# CSV to Spatial Dataframe with ArcGIS Python API
The ArcGIS Python API can also convert CSV file containing coordinates to a spatially enabled dataframe. 


We'll start with the simplest example of creating a point spatial dataframe from a CSV file containing latitude and longitude coordinates. The data we'll use in this exercise is electric vehicle charging locations in North Carolina ([source](https://afdc.energy.gov/data_download)).

## 1. Constructing a Pandas dataframe from the CSV file
We'll use an API to fetch CSV data listing the electric vehicle charging locations in North Carolina and load that file directly into a familiar Pandas dataframe named `df_EVStations`.

In [ ]:
#Import the requests and pandas libraries
import requests
import pandas as pd

In [ ]:
#Construct the request
serviceURL = 'https://developer.nrel.gov/api/alt-fuel-stations/v1.csv'
parameters = {
    'access':'all',
    'api_key':'oA9dHswdtlpAx5qLEdV1StM1mUB8KsgWluSfoEuL',
    'fuel_type':'ELEC',
    'status':'all',
    'state':'NC',
    'download':'true'
}

#Process the request
response = requests.get(serviceURL,parameters)
df_EVStations = pd.read_csv(response.url)

In [ ]:
#Run if the above fails...
#df_EVStations_All = pd.read_csv('./data/alt_fuel_stations (Nov 14 2019).csv',low_memory=False)
#df_EVStations = df_EVStations_All.query('State == "NC"').reset_index()

## 2. Creating geometries from latitude and longitude coordinates
Once again, we begin with a dataframe that includes coordinate data and our task is to conver those coordinates to geometric features. Converting this dataframe to a spatial dataframe is actually easier than in GeoPandas, however the result is not quite the same type of spatially enabled dataframe we constructed with GeoPandas. 

* We begin by importing the ArcGIS GIS API GIS package and creating an anonymous GIS object. 

In [ ]:
#Import the ArcGIS API GIS object
from arcgis import GIS
#Import some other ArcGIS objectgs
from arcgis.features import SpatialDataFrame, GeoAccessor, GeoSeriesAccessor

_When we import the GIS object, we get addtional functionality attached to our Pandas dataframe. If we append a `.spatial` to many dataframe commands, we have a new class of object with new functionality. (More detail [here](https://developers.arcgis.com/python/guide/introduction-to-the-spatially-enabled-dataframe/).) One function is tha ability to convert X and Y columns into geometries..._

* Next we apply the [`from_xy()`](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html#arcgis.features.SpatialDataFrame.from_df) function to the dataframe added by importing the GIS object. 

In [ ]:
#Convert the Pandas dataframe to a spatial dataframe using the `from_xy` function.)
sdf_EVStations = pd.DataFrame.spatial.from_xy(df=df_EVStations,
                                              x_column='Longitude',
                                              y_column='Latitude',
                                              sr=4326)
#What type of object is created?
type(sdf_EVStations)

* We can examine the dataframe's geometry values...

In [ ]:
sdf_EVStations.geometry

* Geometry is stored in the "SHAPE" field...

In [ ]:
sdf_EVStations['SHAPE']

In [ ]:
#Draw
gis = GIS()
theMap = gis.map('North Carolina')
theMap

The GeoSeriesAccessor is a means for working with the set of geometries...

In [ ]:
#Convert the geometry to a GeoSeries Accessor object
geoSeries = GeoSeriesAccessor(sdf_EVStations.geometry)
type(geoSeries)

In [ ]:
#Project the geo
geoSeries_utm = geoSeries.project_as('26917')

In [ ]:
ga = GeoAccessor(sdf_EVStations)

In [ ]:
ga.bbox

We see the obect created is still a Pandas DataFrame. However, we can continue to append `.spatial` to apply spatial functions to the dataset. In fact, if save `sdf_EVStations.spatial` to a variable we see that it's a new object, an ArcGIS ["GeoAccessor"](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html#geoaccessor) object. 

In [ ]:
#Some properties of the GeoAccessor object
gacc_EVStations.bbox

In [ ]:
fc = sdf.spatial.to_feature_collection()

The ArcGIS Python API, when loaded, does an interesting thing with the Pandas dataframe object: it adds a submodule to it such that adding `.spatail` to an existing `Dataframe` command adds a new set of functionality to our dataframe.

In [ ]:
myMap = gis.map('North Carolina')
#myMap.draw(sdf.iloc[0]['SHAPE'])
myMap

In [ ]:
#Import API helper objects: the GeoAccessor and GeoSeriesAccessor
from arcgis.features import GeoAccessor, GeoSeriesAccessor

In [ ]:
sdf = pd.DataFrame.spatial.from_featureclass('./data/HUC12.shp')
type(sdf)

In [ ]:
ga = GeoAccessor(sdf)

In [ ]:
ga.area